In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd

file_paths = [
    "/content/drive/My Drive/legal_data/crpc_qa.json",
    "/content/drive/My Drive/legal_data/ipc_qa.json",
    "/content/drive/My Drive/legal_data/constitution_qa.json"
]

dfs = [pd.read_json(file) for file in file_paths]

print(dfs[0].head())


                                            question  \
0  What is the short title, extent and commenceme...   
1  Which section of the Code of Criminal Procedur...   
2  What does Section 2 of the Code of Criminal Pr...   
3  What is defined in Section 2 of the Code of Cr...   
4  What is the purpose of Section 3 in the Code o...   

                               answer  id  
0  Section 1 of Chapter I PRELIMINARY NaN  
1  Section 1 of Chapter I PRELIMINARY NaN  
2                         Definitions NaN  
3                         Definitions NaN  
4          Construction of references NaN  


In [ ]:
df = pd.concat(dfs,axis=0,ignore_index=True)
df

,question,answer,id
0,"What is the short title, extent and commenceme...",Section 1 of Chapter I PRELIMINARY,NaN
1,Which section of the Code of Criminal Procedur...,Section 1 of Chapter I PRELIMINARY,NaN
2,What does Section 2 of the Code of Criminal Pr...,Definitions,NaN
3,What is defined in Section 2 of the Code of Cr...,Definitions,NaN
4,What is the purpose of Section 3 in the Code o...,Construction of references,NaN
...,...,...,...
14538,Which schedule includes small scale industries...,The Eleventh Schedule includes these industries.,NaN
14539,What welfare programs are included in the Elev...,"Family welfare, Women and child development, S...",NaN
14540,Which Schedule includes programs such as famil...,These programs are included in the Eleventh Sc...,NaN
14541,What does the twelfth schedule of Article 243W...,"Urban planning including town planning, regula...",NaN


In [ ]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes
!pip install sec_api
!pip install -U langchain
!pip install -U langchain-community
!pip install -U sentence-transformers
!pip install -U faiss-gpu

In [ ]:

hf_token = <token>


In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:

model, tokenizer = FastLanguageModel.from_pretrained(

    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
    token = hf_token,
)


==((====))==  Unsloth 2025.1.7: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

Unsloth 2025.1.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
df.fillna('',inplace=True)

<ipython-input-9-d75f31e70ecb>:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('',inplace=True)


In [ ]:

ft_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Below is a user question, Write a response that appropriately answers the question,
include specific details in your response. <|eot_id|>

<|start_header_id|>user<|end_header_id|>

### Question:
{}



<|eot_id|>

### Response: <|start_header_id|>assistant<|end_header_id|>
{}"""

EOS_TOKEN = tokenizer.eos_token


def formatting_prompts_func(examples):
    questions = examples["question"]
    responses  = examples["answer"]
    text = ft_prompt.format(questions, responses) + EOS_TOKEN
    return text
pass

#dataset = load_dataset("virattt/llama-3-8b-financialQA", split = "train")
df['text'] = df.apply(formatting_prompts_func,axis=1)

In [ ]:
df.drop(['id'],axis=1,inplace=True)

In [ ]:
df

,question,answer,text
0,"What is the short title, extent and commenceme...",Section 1 of Chapter I PRELIMINARY,<|begin_of_text|><|start_header_id|>system<|en...
1,Which section of the Code of Criminal Procedur...,Section 1 of Chapter I PRELIMINARY,<|begin_of_text|><|start_header_id|>system<|en...
2,What does Section 2 of the Code of Criminal Pr...,Definitions,<|begin_of_text|><|start_header_id|>system<|en...
3,What is defined in Section 2 of the Code of Cr...,Definitions,<|begin_of_text|><|start_header_id|>system<|en...
4,What is the purpose of Section 3 in the Code o...,Construction of references,<|begin_of_text|><|start_header_id|>system<|en...
...,...,...,...
14538,Which schedule includes small scale industries...,The Eleventh Schedule includes these industries.,<|begin_of_text|><|start_header_id|>system<|en...
14539,What welfare programs are included in the Elev...,"Family welfare, Women and child development, S...",<|begin_of_text|><|start_header_id|>system<|en...
14540,Which Schedule includes programs such as famil...,These programs are included in the Eleventh Sc...,<|begin_of_text|><|start_header_id|>system<|en...
14541,What does the twelfth schedule of Article 243W...,"Urban planning including town planning, regula...",<|begin_of_text|><|start_header_id|>system<|en...


In [ ]:
texts = df['text']
q = df['question']
a = df['answer']


dataset = []

for t,question,answer in zip(texts,q,a):
  dataset.append({'question':question,'answer':answer,'text':t})

dataset[0]

{'question': 'What is the short title, extent and commencement referred to in the Code of Criminal Procedure, 1973?',
 'answer': 'Section 1 of Chapter I PRELIMINARY',
 'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nBelow is a user question, Write a response that appropriately answers the question,\ninclude specific details in your response. <|eot_id|>\n\n<|start_header_id|>user<|end_header_id|>\n\n### Question:\nWhat is the short title, extent and commencement referred to in the Code of Criminal Procedure, 1973?\n\n\n\n<|eot_id|>\n\n### Response: <|start_header_id|>assistant<|end_header_id|>\nSection 1 of Chapter I PRELIMINARY<｜end▁of▁sentence｜>'}

In [ ]:
from datasets import Dataset

hf_dataset = Dataset.from_list(dataset)



In [ ]:
trainer = SFTTrainer(

    model = model,

    tokenizer = tokenizer,

    train_dataset = hf_dataset,

    dataset_text_field = "text",

    max_seq_length = 2048,

    dataset_num_proc = 2,

    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 50,
        learning_rate = 0.0005,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs"

      ),
)


Map (num_proc=2):   0%|          | 0/14543 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 14,543 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 50
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: yobrochill56 (yobrochill56-delhi-technological-university). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
1,4.833700
2,5.124900
3,4.730600
4,3.944600
5,3.525300
6,2.589300
7,2.093100
8,1.698800
9,1.597400
10,1.771000


In [ ]:
# model.save_pretrained("/content/drive/MyDrive/legal_data/deepseek_legal")
# tokenizer.save_pretrained("/content/drive/MyDrive/legal_data/deepseek_legal")

In [ ]:
# Redefining prompt if importing without training
ft_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Below is a user question, paired with retrieved context. Write a response that appropriately answers the question,
include specific details in your response. <|eot_id|>

<|start_header_id|>user<|end_header_id|>

### Question:
{}


### Context:

{}


<|eot_id|>

### Response: <|start_header_id|>assistant<|end_header_id|>
{}"""

if True:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/content/drive/MyDrive/legal_data/deepseek_legal",
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )
    FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.1.7: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/231 [00:00<?, ?B/s]

Unsloth 2025.1.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:

def inference(question,context=""):
  inputs = tokenizer(
  [
      ft_prompt.format(
          question,
          context,
          "",
      )
  ], return_tensors = "pt").to("cuda")


  outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True, pad_token_id=tokenizer.eos_token_id)
  response = tokenizer.batch_decode(outputs)
  return response



def extract_response(resp):

    response_text = resp[0]

    response_marker = "### Response: <|start_header_id|>assistant<|end_header_id|>"


    response_start = response_text.find(response_marker)

    if response_start != -1:

        response_start += len(response_marker)


        eot_index = response_text.find("<｜end▁of▁sentence｜>", response_start)

        if eot_index != -1:

            extracted_response = response_text[response_start:eot_index].strip()
        else:

            extracted_response = response_text[response_start:].strip()

        print("Extracted Response:\n", extracted_response)
    else:
        print("Response marker not found.")
